<a href="https://colab.research.google.com/github/Speedbird45Bravo/rando_projects/blob/main/Torch_61322_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from torch.nn import Module, Sequential, Conv2d, ReLU, MaxPool2d, CrossEntropyLoss, Linear
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import Adam
from datetime import datetime as dt
from pytz import timezone

In [35]:
train = MNIST("/files", train=True, download=True, transform=ToTensor())
test = MNIST("/files", train=False, download=True, transform=ToTensor())
loaders = {"train":DataLoader(train, batch_size=64, shuffle=True, num_workers=4),\
           "test":DataLoader(train, batch_size=64, shuffle=True, num_workers=4)}
tz = timezone("US/Eastern")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [36]:
class NeuralNetwork(Module):

  def __init__(self):

    super(NeuralNetwork, self).__init__()

    self.c1 = Sequential(Conv2d(1,16,5,1,2),ReLU(),MaxPool2d(2))
    self.c2 = Sequential(Conv2d(16,32,5,1,2),ReLU(),MaxPool2d(2))
    self.out = Sequential(Linear(32 * 7 * 7, 10))
  
  def forward(self, x):
    x = self.c1(x)
    x = self.c2(x)
    x = x.view(x.size(0),-1)
    output = self.out(x)
    return output, x

In [37]:
cnn = NeuralNetwork()
print(f"Network printed at {dt.now(tz=tz)}")
print(cnn)

Network printed at 2022-06-13 11:48:39.310472-04:00
NeuralNetwork(
  (c1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (c2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Linear(in_features=1568, out_features=10, bias=True)
  )
)


In [38]:
epochs = 6
loss_function = CrossEntropyLoss()
rabinowitz = Adam(cnn.parameters(), lr=0.01)

In [42]:
def train(cnn, epochs, loaders):

  cnn.train()

  for epoch in range(epochs):

    for i, (images, labels) in enumerate(loaders["train"]):
      b_X = Variable(images)
      b_y = Variable(labels)
      output = cnn(b_X)[0]
      loss = loss_function(output, b_y)

      rabinowitz.zero_grad()

      loss.backward()

      rabinowitz.step()

      if (i + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item()}")

print(f"Model Trained at {dt.now(tz=tz)}")

Model Trained at 2022-06-13 11:49:06.381397-04:00


In [43]:
train(cnn, epochs, loaders)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1/6 | Loss: 0.020855560898780823
Epoch 1/6 | Loss: 0.12698858976364136
Epoch 1/6 | Loss: 0.22904489934444427
Epoch 1/6 | Loss: 0.006210739724338055
Epoch 1/6 | Loss: 0.3023676574230194
Epoch 1/6 | Loss: 0.07584307342767715
Epoch 1/6 | Loss: 0.21356087923049927
Epoch 1/6 | Loss: 0.12296122312545776
Epoch 1/6 | Loss: 0.0026560567785054445
Epoch 2/6 | Loss: 0.05857837200164795
Epoch 2/6 | Loss: 0.03735177218914032
Epoch 2/6 | Loss: 0.021476145833730698
Epoch 2/6 | Loss: 0.029289785772562027
Epoch 2/6 | Loss: 0.09728538990020752
Epoch 2/6 | Loss: 0.053849879652261734
Epoch 2/6 | Loss: 0.1355440765619278
Epoch 2/6 | Loss: 0.05806760862469673
Epoch 2/6 | Loss: 0.009670156054198742
Epoch 3/6 | Loss: 0.013206083327531815
Epoch 3/6 | Loss: 0.0725955069065094
Epoch 3/6 | Loss: 0.023382266983389854
Epoch 3/6 | Loss: 0.051365263760089874
Epoch 3/6 | Loss: 0.05498507246375084
Epoch 3/6 | Loss: 0.1466871052980423
Epoch 3/6 | Loss: 0.002601295942440629
Epoch 3/6 | Loss: 0.007445058319717646
Epo

In [61]:
def test():

  cnn.eval()

  with torch.no_grad():

    correct = 0
    total = 0

    for images, labels in loaders["test"]:
      test_output, last_layer = cnn(images)
      y_pred = torch.max(test_output,1)[1].data.squeeze()
      accuracy = ((y_pred == labels).sum().item()) / float(labels.size(0))
      pass

    print(f"Test Accuracy: {accuracy * 100}%")
    print(f"Model Tested at {dt.now(tz=tz)}")

In [63]:
test()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Accuracy: 93.75%
Model Tested at 2022-06-13 12:01:04.570973-04:00
